문제 2-4: 여행 계획 분석기

In [5]:
import os
from pprint import pprint
from dotenv import load_dotenv
from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# .env 파일에서 환경 변수 로드
load_dotenv()

# LLM 설정 (Upstage Solar 모델 사용)
llm = ChatUpstage(
    api_key=os.getenv("UPSTAGE_API_KEY"),
    model="solar-1-mini-chat", # solar-1-mini-chat-ko-instruct 모델도 사용 가능
    temperature=0.1
)

# 1. 추출할 정보의 스키마(구조) 정의
response_schemas = [
    ResponseSchema(name="destination", description="여행지", type="string"),
    ResponseSchema(name="duration", description="여행 기간", type="string"),
    ResponseSchema(name="budget", description="총 예산", type="string"),
    ResponseSchema(name="rating", description="추천도 (1-5점)", type="string"),
    ResponseSchema(name="activities", description="주요 활동 목록", type="list"),
]

# 2. StructuredOutputParser 생성
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# 3. 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(
    """주어진 여행 후기 텍스트에서 다음 정보를 추출해주세요.
    
{instructions}

여행 후기:
{review}"""
)

# 4. 체인 생성 및 실행
chain = prompt | llm | output_parser
review_text = "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."
result = chain.invoke({"review": review_text, "instructions": format_instructions})

print("## 입력 텍스트:")
pprint(review_text)
print("\n## 추출된 구조화 데이터:")
pprint(result)

## 입력 텍스트:
('지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, '
 '감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요.')

## 추출된 구조화 데이터:
{'activities': ['해운대에서 바다구경', '자갈치시장에서 회 먹기', '감천문화마을 구경'],
 'budget': '30만원',
 'destination': '부산',
 'duration': '2박 3일',
 'rating': '4점'}
